In [1]:
import pandas as pd
import numpy as np 

In [2]:
import os

In [3]:
from imutils import paths

In [6]:
raw_data_path = "./data/"

In [7]:
import torch

In [8]:
from skimage import io, transform

### Sample submission file

In [9]:
df_sample_sub = pd.read_csv(f"{raw_data_path}sample_submission.csv")
df_sample_sub.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


### Labels

In [10]:
df_sample_sub = pd.read_csv(f"{raw_data_path}labels.csv")
df_sample_sub.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [11]:
from PIL import Image

In [12]:
from torch.utils.data import Dataset

In [13]:
class DogBreedsDataset(Dataset):
    def __init__(self, dogbreed_csv, dogbreed_images, transform=None):
        self.samples = []
        self.df = pd.read_csv(dogbreed_csv)
        self.dogbreed_images = dogbreed_images
        self.images_list = list(paths.list_images(dogbreed_images))
        self.mapping = dict(zip(self.df["breed"].unique(), range(0, len(self.df["breed"].unique()))))
        self.df["breed"] = self.df["breed"].map(self.mapping)
        self.transform = transform
        
    def get_image_from_folder(self, image_name):
        return os.path.join(self.dogbreed_images, image_name)
    
    def get_mapping(self):
        return self.mapping
        
    def __len__(self):
        if len(self.images_list) == len(self.df):
            return len(self.images_list)
        else:
            return "mismatch between lengths in image folder and csv containing labels"

    def __getitem__(self, idx):
        image = io.imread(self.images_list[idx])
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        label = self.df[self.df["id"] == self.images_list[idx].split("/")[-1].split(".")[0]]["breed"].as_matrix().astype("float")
        label = [int(label) for x in label]
#         label = np.asarray(label)
#         label = torch.from_numpy(label)
        label = torch.LongTensor(label)
        return image, label

In [14]:
from torchvision import transforms

In [15]:
img_transforms = transforms.Compose([
    transforms.Resize((64, 64)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.405], 
                    std = [0.229, 0.224, 0.225])
])

In [16]:
dataset = DogBreedsDataset("./data/labels.csv", "./data/train/", transform=img_transforms)
print(len(dataset))
print(dataset[1000])

10222
(tensor([[[-0.7650, -0.7137, -0.6794,  ..., -0.4911, -0.5253, -0.5596],
         [-0.7308, -0.6965, -0.6623,  ..., -0.5253, -0.5424, -0.5767],
         [-0.7137, -0.6794, -0.6452,  ..., -0.6452, -0.6623, -0.6794],
         ...,
         [-1.0048, -1.1247, -1.0390,  ..., -0.9705, -1.0562, -0.9192],
         [-1.0904, -1.1075, -1.1075,  ..., -1.0048, -0.9877, -0.8164],
         [-0.8849, -1.0219, -1.1589,  ..., -0.9877, -0.9020, -0.9192]],

        [[-0.7577, -0.7052, -0.6702,  ..., -0.5651, -0.5826, -0.6176],
         [-0.7227, -0.6877, -0.6527,  ..., -0.5476, -0.5826, -0.6001],
         [-0.7052, -0.6702, -0.6352,  ..., -0.5826, -0.6001, -0.6001],
         ...,
         [-0.4776, -0.5826, -0.4776,  ..., -0.5476, -0.5651, -0.4776],
         [-0.5651, -0.6001, -0.5826,  ..., -0.6001, -0.4951, -0.2850],
         [-0.3550, -0.4951, -0.6176,  ..., -0.5826, -0.4076, -0.3725]],

        [[-0.4580, -0.4057, -0.3708,  ..., -0.3011, -0.3534, -0.3882],
         [-0.4231, -0.3882, -0.3534,  

/Users/sachitanandp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [18]:
from skimage import io, transform

In [19]:
from torch.utils import data

In [20]:
batch_size=64

In [21]:
train_data = data.DataLoader(dataset, batch_size=batch_size)

In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [23]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 120)
        
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [24]:
simplenet = SimpleNet()

In [25]:
import torch.optim as optim

In [26]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [27]:
import torch

In [28]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=120, bias=True)
)

In [29]:
def train(model, optimizer, loss_fn, train_loader, epochs=10, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0 
        num_correct = 0
        num_examples = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device).reshape(inputs.shape[0])
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()*inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(outputs), dim=1)[1], labels).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        training_loss /= len(train_loader.dataset)
        
        print('Epoch:{}, training_loss:{:.2f}, accuracy={:.2f}'\
              .format(epoch, training_loss, num_correct/num_examples))
            

In [98]:
train(simplenet, optimizer, nn.CrossEntropyLoss(), train_data, epochs=3, device=device)

/Users/sachitanandp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/sachitanandp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:0, training_loss:4.51, accuracy=0.03
Epoch:1, training_loss:4.40, accuracy=0.03
Epoch:2, training_loss:4.31, accuracy=0.04
